# Manage Privileges in Unity Catalog (Object Level Security)

**Welcome back to the Databricks Zero to Hero Series!**

In this notebook, we will explore **Object Level Security** in Unity Catalog. We will learn how to manage privileges (permissions) for different securable objects like Catalogs, Schemas, Tables, and Volumes.

### Learning Objectives
1.  **Understand the Object Model:** Metastore -> Catalog -> Schema -> Table/Volume.
2.  **Permission Models:**
    *   **Top-Down (Inheritance):** Granting access at a higher level (Parent) automatically applies to lower levels (Children).
    *   **Bottom-Up (Selective):** Granting access to specific children while providing minimal traversal access to parents.
3.  **Key Privileges:** `USE CATALOG`, `USE SCHEMA`, `SELECT`, `BROWSE`, `READ VOLUME`, `MANAGE`.
4.  **Managing via SQL:** Using `GRANT`, `REVOKE`, and `SHOW GRANTS` commands.

### Prerequisites
*   You must be a **Metastore Admin** or an owner of the object to grant permissions.
*   A workspace enabled with Unity Catalog.
*   Users or Groups created (e.g., `data_engineers`, `data_analysts`) to test permissions.

## 1. The Unity Catalog Object Model & Inheritance

Unity Catalog follows a hierarchy:
`Metastore` > `Catalog` > `Schema` > `Table / View / Volume / Function`

### How Inheritance Works
*   **Inherited Permissions:** If you grant a privilege (e.g., `SELECT`) on a **Catalog**, the user automatically gets `SELECT` on **all Schemas** in that catalog and **all Tables** in those schemas.
*   **Explicit Permissions:** You can also grant permissions on a specific table directly.

Let's start by checking permissions at the highest level: the **Metastore**.

In [ ]:
-- Check who has permissions on the Metastore
-- Note: Usually only Metastore Admins have access here.
SHOW GRANTS ON METASTORE;

## 2. Metastore Level Privileges: Creating Catalogs

By default, normal users cannot create Catalogs. Only Metastore Admins can. If you want a specific team (e.g., Data Engineers) to be able to create their own catalogs, you must grant them the `CREATE CATALOG` privilege on the Metastore.

*(Note: Replace `data_engineers` with your actual group or user name)*

In [ ]:
-- Grant permission to a group to create new Catalogs in this Metastore
GRANT CREATE CATALOG ON METASTORE TO `data_engineers`;

-- Verify the grant
SHOW GRANTS ON METASTORE;

## 3. Top-Down Approach (Bulk Access)

This approach is best when you want to provide **broad access**. For example, giving Data Analysts read access to **everything** inside the `dev` catalog.

**Required Permissions:**
To access a table `dev.bronze.customers`, a user needs:
1.  `USE CATALOG` on `dev`
2.  `USE SCHEMA` on `bronze`
3.  `SELECT` on `customers`

In a Top-Down approach, we apply these at the **Catalog** level, and they trickle down.

In [ ]:
-- Let's create a dummy catalog for testing permissions if it doesn't exist
CREATE CATALOG IF NOT EXISTS dev_catalog;
CREATE SCHEMA IF NOT EXISTS dev_catalog.bronze;
CREATE TABLE IF NOT EXISTS dev_catalog.bronze.sample_table (id INT, name STRING);

-- Grant Bulk Access:
-- 1. Allow traversing the catalog (USE CATALOG)
-- 2. Allow traversing all schemas (USE SCHEMA - inherited)
-- 3. Allow reading all tables (SELECT - inherited)
GRANT USE CATALOG, USE SCHEMA, SELECT ON CATALOG dev_catalog TO `data_analysts`;

-- Check grants on the Catalog
SHOW GRANTS ON CATALOG dev_catalog;

### Verifying Inheritance
If we check the grants on the *Schema* or *Table* inside `dev_catalog`, we won't see an explicit row saying `data_analysts` has access. Instead, the UI would show "Inherited from Catalog". In SQL, `SHOW GRANTS` generally shows explicit grants, but the access is effective.

In [ ]:
-- Revoking Access (Cleanup)
-- If we want to remove this bulk access:
REVOKE ALL PRIVILEGES ON CATALOG dev_catalog FROM `data_analysts`;

## 4. Bottom-Up Approach (Selective Access)

This approach is for **Fine-Grained Access Control**.
Scenario: You want a user to read **ONLY** `dev_catalog.bronze.sample_table` but **NOT** other tables in the same schema.

**The Rule of Traversal:**
Even if you grant `SELECT` on the specific table, the user cannot reach it unless they have "Usage" permission on the parent containers.

1.  Grant `SELECT` on the specific **Table**.
2.  Grant `USE SCHEMA` on the parent **Schema**.
3.  Grant `USE CATALOG` on the parent **Catalog**.

In [ ]:
-- Step 1: Grant SELECT on the specific table only
GRANT SELECT ON TABLE dev_catalog.bronze.sample_table TO `data_analysts`;

-- Step 2: Grant permission to enter the Schema (but not read other tables in it)
GRANT USE SCHEMA ON SCHEMA dev_catalog.bronze TO `data_analysts`;

-- Step 3: Grant permission to enter the Catalog
GRANT USE CATALOG ON CATALOG dev_catalog TO `data_analysts`;

## 5. The BROWSE Privilege

Sometimes you want users to see **that data exists** (metadata visibility, lineage) without being able to query the actual data. This is what `BROWSE` is for.

*   `BROWSE`: Allows viewing metadata (Catalog Explorer lists it) but querying fails unless `SELECT` is also granted.

In [ ]:
-- Grant Browse permission on the catalog so users can see it in the UI
GRANT BROWSE ON CATALOG dev_catalog TO `account_users`;

## 6. Managing Volumes

Volumes (for unstructured data) have their own set of permissions:
*   `READ VOLUME`: Read files.
*   `WRITE VOLUME`: Upload/Delete files.

In [ ]:
-- Create a volume for demo
CREATE VOLUME IF NOT EXISTS dev_catalog.bronze.my_files;

-- Grant Read access to the volume
-- Remember: User still needs USE CATALOG and USE SCHEMA on parents
GRANT READ VOLUME ON VOLUME dev_catalog.bronze.my_files TO `data_engineers`;

-- Grant Write access
GRANT WRITE VOLUME ON VOLUME dev_catalog.bronze.my_files TO `data_engineers`;

## 7. ownership and MANAGE Privilege

*   **Owner:** The creator of an object is its owner. Owners have all privileges.
*   **MANAGE Privilege:** This is a very powerful privilege. It allows a user to modify the object, drop it, or change permissions for others. It essentially creates a "Delegate Owner".

**Caution:** Be careful when granting `MANAGE` or `ALL PRIVILEGES`.

In [ ]:
-- Grant full control over a table to a specific user (Dangerous if not intended)
GRANT MANAGE ON TABLE dev_catalog.bronze.sample_table TO `data_engineers`;

In [ ]:
# Clean up resources used in this specific demo
# spark.sql("DROP CATALOG IF EXISTS dev_catalog CASCADE")
print("Permissions demo completed.")